In [120]:
import owncloud
import pandas as pd
import numpy as np
import smtplib
import ssl
from email.message import EmailMessage
import unidecode
import random
import string
import os

users = pd.read_csv('Mitgliederliste_CSV.csv')

The settings cell: this is where you input your email client and OwnCloud parameters, as well as some additional info. 

**username_connector** is used to remove conjuntions from the usernames of accounts shared by several people. For example, if it is set to *' and '*, the username *'Ursula and Fiona'* will be formated as *'UrsulaFiona'*.

**email_subject** sets the subject of the email that you'll be sending out.

In [1]:
port = 587
smtp_server = os.environ.get('MISCHA_BOTMAIL_SMTP')
password = os.environ.get('MISCHA_BOTMAIL_PASS')
sender_mail = os.environ.get('MISCHA_BOTMAIL_USER')

oc = owncloud.Client(os.environ.get('MISCHA_OWNCLOUD_URL'))
oc.login(os.environ.get('NESTOR_OWNCLOUD_USER'), os.environ.get('NESTOR_OWNCLOUD_PASS'))

username_connector = ' und '

email_subject = 'NextCloud Account'

The main function of the script, does all the work aside from creating OwnCloud Groups. Note that the cell only defines the function and does not execute it.

In [124]:
# 

def Make_Notify(csv):
    Total_list = []
    for index, row in csv.iterrows():

        # Formats usernames in the following ways:
        # 1. Removes spaces
        # 2. Removes the connector string for accounts used by multiple people
        # 3. Removes any inconvenient accented characters
        # 4. Appends the apartment number

        if not pd.isna(row['Vorname']):
            if not username_connector in row['Vorname']:
                Usersafe = row['Vorname'].replace(' ', '')
            else:
                Usersafe = row['Vorname'].replace(username_connector, '')
            Usersafe += str(row['TopNr.'])
            Usersafe = unidecode.unidecode(Usersafe)

        # Generates a random temporary 5 digit PIN for every user and logs it in a .txt file

        if not pd.isna(row['E-Mail']):
            UserPass = ''.join(random.choice(string.ascii_uppercase + string.digits) for i in range(8))
            logstring = Usersafe + ' ' + UserPass + ' ' + row['E-Mail'] + '\n'
            logfile = open('Log.txt', 'a')
            logfile.write(logstring)
            logfile.close()
            
            # Builds the email header and body

            msg = EmailMessage()
            msg['Subject'] = email_subject
            msg['From'] =  sender_mail
            msg['To'] = row['E-Mail']

            mail_text = f'Ein Mischa Nextcloud Account wurde für dich erstellt! Du kannst dich unter {os.environ.get('MISCHA_OWNCLOUD_URL')} mit den folgenden Daten einloggen: \nUsername: {Usersafe} \nKennwort: {UserPass} \nDas Kennwort wurde nur für die erste Benutzung generiert. Bitte log dich ein und ändere das Passwort. \nDiese E-Mail wurde automatisch generiert. Falls du Fragen hast, poste sie bitte ins Slack. \n\n---\n\nA Mischa Nextcloud account has been created for you. You can log in at https://mehr.mischa.wien using the following credentials: \nUsername: {Usersafe} \nPassword: {UserPass} \nThe password is meant to be used only once, the first time you log in. Please change it as soon as possible.\nThis E-Mail was automatically generated. In case you have any questions, please post them in our Slack.'
            msg.set_content(mail_text)
            server.send_message(msg)

            # Checks if the user exists and they do not, makes an accound and adds it to each group listed

            if not oc.user_exists(Usersafe):
                oc.create_user(Usersafe, UserPass)

            if not pd.isna(row['TG']):
                TGuser = str(row['TG']).split(', ')
            
                for TG in TGuser:
                    oc.add_user_to_group(Usersafe, TG)

This cell creates the groups present in the CSV, without adding any users to them.

In [42]:
# Gets list of all Groups

TGlist = []

for index, row in users.iterrows():
        TGuser = str(row['TG']).split(', ')
        if TGuser != ['nan']:
            for element in TGuser:
                    if element not in TGlist:
                        TGlist.append(element)
print(TGlist)

# Creates the Groups

for element in TGlist:
    if not oc.group_exists(element):
        oc.create_group(element)

['TG Hochbeetli', 'TG Technik', 'TG Workshops', 'TG Feste Werkstatt', 'TG Fitness', 'TG Küche', 'TG FairPlay', 'TG Freiraum', 'TG Heimkino', 'TG Website', 'TG Kooperationsvereinbarung']


The cell below connects to the email server, then executes the main Make_Notify() function, creating a Nextcloud account for each person in the CSV and sending them a notification. Running it will take longer based on the amount of users you're making.

In [125]:
context = ssl.create_default_context()

try:
    server = smtplib.SMTP(smtp_server, port)
    server.ehlo()
    server.starttls(context=context)
    server.ehlo()
    server.login(sender_mail, password)

    #Send message here
    Make_Notify(users)

except Exception as e:
    print(e)
finally:
    server.quit()